In [38]:
import numpy as np
import scipy
from scipy import signal
import mne
import glob
from sklearn.decomposition import PCA
import csv

from utils.logger import log_result
from utils import experiments

#### Calculate the band power over a time series

In [39]:
# Calculate the band power over a time series

def bandpower(x, fs, fmin, fmax):
    """
    Returns the band power over the specified frequency interval
    
    x - input time series (1D array)
    fs - sampling frequency
    fmin - min frequency
    fmax - max frequency
    """
    
    f, Pxx = signal.periodogram(x, fs=fs)
    ind_min = scipy.argmax(f > fmin) - 1
    ind_max = scipy.argmax(f > fmax) - 1
    return scipy.trapz(Pxx[ind_min: ind_max], f[ind_min: ind_max])

#### Reduce the dimensionality of an array to the specified number of components

In [40]:
def pca(array, n_components=5):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(array)

# Apply bandpower to datasets

In [41]:
def get_datasets(patient_type_location, recording_type_expression):
    """
    Returns relevant datasets (f.e. all right-hand recordings of patients with pain) as a list of np arrays
    First parameter should be a regex for location, second parameter should be a regex for dataset type
    E.g. if right-hand movement datasets of patients with pain are in /data/pp/ and their file names contain '_RH_'
    then patient_type_location=/data/pp/ and recording_type_expression='_RH_'
    """
    
    # Find locations of matching dataset files
    if recording_type_expression != l_new:
        sets_locations = glob.glob(patient_type_location + recording_type_expression + suffix)
    else:
        # For the newer (PDP/PP) dataset we had to use a separate expression with includes the file extension
        sets_locations = glob.glob(patient_type_location + recording_type_expression)
    
    sets = []
    for path in sets_locations: 
        sets.append(mne.io.read_epochs_eeglab(path))
        
    return np.array(np.array([(patient._data) for patient in sets]))

In [43]:
# Calculate bandpower for all channels for a patient
bands = [(4, 8), (8, 13), (13, 30)]
time_series_index = range(1250)[:]

def channels_bandpower(channels, bands, fs=250):
    b = bandpower
    return np.array(list(map( lambda arr: [b(arr[time_series_index], fs, band[0], band[1]) for band in bands], channels)))

#### Define dataset locations and expressions

In [44]:
root = './../../../'
suffix = '*.set'

# Old (PP/PNP datasets)
location_healthy = root + 'data/raw/HV/*/'
location_pain = root + 'data/raw/PP/*/'
location_nopain = root + 'data/raw/PnP/*/'

# New (PDP/PNP datasets)
location_pwp = root + 'data_new/raw/PwP/*/'
location_pdp = root + 'data_new/raw/PdP/*/'
location_pnp = root + 'data_new/raw/PnP/*/'

rh = '*_RH*'
lh = '*_LH*'
l_new = '*_L.set'   # NO SUFFIX
l_old = '*_L_*'


# As an example, get paths of all PP/PNP datasets from right-hand movements
sets_healthy_rh = glob.glob(location_pain + rh + suffix)
sets_healthy_rh

['./../../../data/raw/PP\\PP1\\PP1_F1_RH_Removed_ICA.set',
 './../../../data/raw/PP\\PP10\\PP10_F1_RH_Removed_ICA.set',
 './../../../data/raw/PP\\PP11\\PP11_F1_RH_Removed_ICA.set',
 './../../../data/raw/PP\\PP2\\PP2_F1_RH_Removed_ICA.set',
 './../../../data/raw/PP\\PP3\\PP3_F1_RH_Removed_ICA.set',
 './../../../data/raw/PP\\PP4\\PP4_F1_RH_Removed_ICA.set',
 './../../../data/raw/PP\\PP5\\PP5_F1_RH_Removed_ICA.set',
 './../../../data/raw/PP\\PP6\\PP6_F1_RH_Removed_ICA.set',
 './../../../data/raw/PP\\PP7\\PP7_F1_RH_Removed_ICA.set',
 './../../../data/raw/PP\\PP9\\PP9_F1_RH_Removed_ICA.set']

#### Now read the chosen datasets

In [71]:
pp_rh_raw = get_datasets(location_pain, rh)
pnp_rh_raw = get_datasets(location_nopain, rh)

Extracting parameters from ./../../../data/raw/PP\PP1\PP1_F1_RH_Removed_ICA.set...
57 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../../data/raw/PP\PP10\PP10_F1_RH_Removed_ICA.set...
57 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../../data/raw/PP\PP11\PP11_F1_RH_Removed_ICA.set...
59 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../../data/raw/PP\PP2\PP2_F1_RH_Removed_ICA.set...
54 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../../data/raw/PP\PP3\PP3_F1_RH_Removed_ICA.set...
51 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting param

In [72]:
# The entry for a patient should have shape (n_repetitions, n_channels, n_readings)
pp_rh_raw[4].shape

(51, 61, 1250)

#### Apply the bandpower 

In [73]:
pp_rh_bp = np.array([np.array([channels_bandpower(repetition, bands) for repetition in patient]) for patient in pp_rh_raw])
pnp_rh_bp = np.array([np.array([channels_bandpower(repetition, bands) for repetition in patient]) for patient in pnp_rh_raw])

In [74]:
# Get the total number of repetitions for each class
pp_count = np.vstack(pp_rh_bp).shape[0]
pnp_count = np.vstack(pnp_rh_bp).shape[0]
pnp_count

473

In [75]:
pnp_rh_bp[0].shape

(50, 61, 3)

#### Concatenate the two classes

In [76]:
pp_and_pnp_bp = np.concatenate((pp_rh_bp, pnp_rh_bp))
pp_and_pnp_bp.shape

(19,)

#### Set some patients aside for testing

In [77]:
def test_setup(test_index, total_size):
    """
    Returns a pair consisting of boolean (True is test patient is PP) and test label
    Labels are 1 for pain, 0 for no pain
    """
    
    test_is_pp = test_index < len(pp_rh_bp)
    test_label = 1 if test_is_pp else 0
    return test_is_pp, test_label


def get_train_test(data, test_index):
    """
    Splits into train and test sets based on the index of the test patient
    Returns pair of test and train
    """
    
    return data[test_index], np.delete(data, test_index)


def get_pp_pnp_length(pp_count, pnp_count, test_count, test_is_pp):
    """ Returns pair of the lengths of PP train data and respectively PNP train data """
    
    pp_train_len = pp_count if not test_is_pp else pp_count - test_count
    pnp_train_len = pnp_count if test_is_pp else pnp_count - test_count
    return pp_train_len, pnp_train_len


def ravel_all_trials(data, channels):
    """
    Ravel first dimention so that trials from all patients are treated separately; select channels
    """
    return np.array(list(map(np.ravel, data[:, channels, :])))

In [78]:
test_index = 4

test_is_pp, test_label = test_setup(test_index, len(pp_rh_bp))
test_label

1

In [79]:
test_p, train_p = get_train_test(pp_and_pnp_bp, test_index)
test_p.shape

(51, 61, 3)

In [80]:
train_p_separated = np.vstack(train_p)
train_p_separated.shape

(952, 61, 3)

In [55]:
pp_train_len, pnp_train_len = get_pp_pnp_length(pp_count, pnp_count, len(test_p), test_is_pp)
pp_train_len

479

Define a multiplier for all features - if 1, then input data is unaltered

In [56]:
mul = 1

In [57]:
selected_channels = [10, 11]

In [58]:
train = ravel_all_trials(train_p_separated, selected_channels) * mul
train.shape

(952, 6)

In [59]:
test = ravel_all_trials(test_p, selected_channels) * mul
test.shape

(51, 6)

#### Define the number of PCA components we will use

In [60]:
n_components = 3

In [61]:
pca(train, n_components).shape

(952, 3)

#### Generate labels

In [62]:
labels = [1] * pp_train_len + [0] * pnp_train_len
test_labels = [test_label] * len(test)

#### It's time to learn

In [63]:
from sklearn import neighbors, svm
from sklearn.model_selection import train_test_split

In [64]:
# Try a simple KNN classification

knn = neighbors.KNeighborsClassifier(n_neighbors=3)
x_train, x_test, y_train, y_test = train_test_split(train, labels, test_size=0.05)
knn.fit(x_train, y_train)
knn.score(x_train, y_train)
knn.score(x_test, y_test)

0.9166666666666666

In [65]:
knn.score(test, [test_label]*len(test))

0.21568627450980393

In [66]:
np.count_nonzero(knn.predict(test) == test_labels)/len(test)

0.21568627450980393

## Classification with cross validation

### KNN

In [73]:
def classify_knn_with_xvalid(data_pp_bp, data_pnp_bp, n_neighbours, selected_channels, test_index, mul, verbose=True):
    """
    Leaves out patient at test_index and trains a KNN classifier on all other patients
    Validates classifier on patient at test_index
    Returns accuracy over all repetitions for the test patient
    """
    
    data_bp = np.concatenate((data_pp_bp, data_pnp_bp))
    
    test_is_pp, test_label = test_setup(test_index, len(data_pp_bp))
    test_p, train_p = get_train_test(data_bp, test_index)
    train_p_separated = np.vstack(train_p)
    pp_train_len, pnp_train_len = get_pp_pnp_length(pp_count, pnp_count, len(test_p), test_is_pp)
    
    # Apply PCA for dimensionality reduction as defined above
    train = pca(ravel_all_trials(train_p_separated, selected_channels) * mul, n_components=n_components)
    test = pca(ravel_all_trials(test_p, selected_channels) * mul, n_components=n_components)
    
    labels = [1] * pp_train_len + [0] * pnp_train_len
    test_labels = [test_label] * len(test)
    
    if verbose:
        print('Test index', test_index, 'Testing on patient with', len(test), 'repetitions.')
    
    clas = neighbors.KNeighborsClassifier(n_neighbors=n_neighbours)
    clas.fit(train, labels)
    train_acc = clas.score(train, labels)
    test_acc = clas.score(test, test_labels)
    
    if verbose:
        print('Train score:', train_acc, '  Test score:', test_acc)
    
    return test_acc
    
    

In [74]:
classify_knn_with_xvalid(pp_rh_bp, pnp_rh_bp, 23, [0, 3, 10, 36], 2, 10000000000000)

Test index 2 Testing on patient with 59 repetitions.
Train score: 0.7966101694915254   Test score: 0.4406779661016949


0.4406779661016949

#### Cross validate over the whole dataset

In [75]:
total_score = 0
patients_correct = 0
for i in range(len(pp_and_pnp_bp)):
    score = classify_knn_with_xvalid(pp_rh_bp, pnp_rh_bp, 9, [5, 16, 24, 36, 60], i, mul)
    total_score += score
    if score > 0.5:
        patients_correct += 1
    
print('Overall accuracy', total_score/len(pp_and_pnp_bp))
print('Correctly labeled', patients_correct, 'out of', len(pp_and_pnp_bp))

Test index 0 Testing on patient with 57 repetitions.
Train score: 0.7790697674418605   Test score: 0.9122807017543859
Test index 1 Testing on patient with 57 repetitions.
Train score: 0.7917547568710359   Test score: 0.9824561403508771
Test index 2 Testing on patient with 59 repetitions.
Train score: 0.7510593220338984   Test score: 0.3220338983050847
Test index 3 Testing on patient with 54 repetitions.
Train score: 0.7744994731296101   Test score: 0.6111111111111112
Test index 4 Testing on patient with 51 repetitions.
Train score: 0.7941176470588235   Test score: 1.0
Test index 5 Testing on patient with 58 repetitions.
Train score: 0.8021164021164021   Test score: 0.8275862068965517
Test index 6 Testing on patient with 56 repetitions.
Train score: 0.8046462513199577   Test score: 1.0
Test index 7 Testing on patient with 32 repetitions.
Train score: 0.8105046343975283   Test score: 1.0
Test index 8 Testing on patient with 52 repetitions.
Train score: 0.804416403785489   Test score: 0.9

#### Cross validate over multiple channels

We use this to check which channels yield better results

In [76]:
# Log results in a csv file
file = open('all_results/test.csv', 'a', newline='')
name = 'Bandpower + PCA + KNN'
notes = 'freq bands 4-8,8-13,13-30, k='
notes_c = ', n_components='

previous_channels = []
k = 19

max_acc = {'index': 0, 'value': 0}
for channel in range(61):    
    total_score = 0
    correct_patients = 0
    for i in range(len(pp_and_pnp_bp)):
        score = classify_knn_with_xvalid(pp_rh_bp, pnp_rh_bp, k, previous_channels + [channel], i, mul, verbose=False)
        total_score += score
        if score > 0.5:
            correct_patients += 1
        
    avg_score = total_score/len(pp_and_pnp_bp)
    print(channel, avg_score, correct_patients)
    
    log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'RH', str(previous_channels + [channel]), notes + str(k) + notes_c + str(n_components))
    
    if avg_score > max_acc['value']:
        max_acc['index'] = channel
        max_acc['value'] = avg_score
        
file.close()
        
print('Max accuracy:', max_acc['index'], max_acc['value'])

0 0.43420301373444276 4
1 0.3940722792469556 6
2 0.4839876033874894 10
3 0.43574237659561116 9
4 0.4175584036193553 8
5 0.4211539121757564 8
6 0.40926323103658424 8
7 0.42670350660701134 8
8 0.4898770266642892 11
9 0.42741570612095087 8
10 0.4269108187798073 8
11 0.4446211972808326 8
12 0.43944660624654075 10
13 0.4369341860318163 9
14 0.42801395997353064 7
15 0.4238064229931583 8
16 0.4348907421590404 9
17 0.46892721052628994 11
18 0.44173993113197807 9
19 0.4193551351745775 9
20 0.41678713337638557 8
21 0.4058881813414496 7
22 0.4222684956064862 9
23 0.41460126184117396 8
24 0.42073875367182284 9
25 0.4603265204875383 5
26 0.476384859495798 9
27 0.4049864591960903 9
28 0.41530172792174724 10
29 0.4075430221114767 8
30 0.43180377311706125 9
31 0.448729877189854 8
32 0.41945741447795504 7
33 0.4410620639655514 9
34 0.4395477580724258 8
35 0.4291558694505733 8
36 0.4819196244493854 9
37 0.45269399678915423 9
38 0.453800635610774 9
39 0.4174689180255331 8
40 0.4357220417069145 8
41 0.385

### Cross validate over multiple n_neighbours

In [96]:
file = open('all_results/bandpower_pca_knn_results.csv', 'a', newline='')
name = 'Bandpower + PCA + KNN'
notes = 'freq bands 4-8,8-13,13-30, k='
notes_c = ', n_components='

channels = [8]

max_acc = {'index': 0, 'value': 0}
for n_neighbours in range(1, 100, 5):    
    total_score = 0
    correct_patients = 0
    for i in range(len(pp_and_pnp_bp)):
        score = classify_knn_with_xvalid(pp_rh_bp, pnp_rh_bp, n_neighbours, channels, i, mul, verbose=False)
        total_score += score
        if score > 0.5:
            correct_patients += 1
        
    avg_score = total_score/len(pp_and_pnp_bp)
    print(n_neighbours, avg_score, correct_patients)
    
    log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'RH', str(channels), notes + str(n_neighbours) + notes_c + str(n_components))
    
    if avg_score > max_acc['value']:
        max_acc['index'] = n_neighbours
        max_acc['value'] = avg_score
        
file.close()
        
print('Max accuracy:', max_acc['index'], max_acc['value'])

1 0.5136758819146736 9
6 0.4785311886755917 9
11 0.48965425136444224 11
16 0.4937298620472069 11
21 0.49126711494484204 11
26 0.48407435339847965 11
31 0.4875966262529175 11
36 0.48109807174234204 11
41 0.4801769571049895 11
46 0.4696324291032629 10
51 0.4712037660398563 11
56 0.47144413406415764 10
61 0.4683555016129158 10
66 0.46640973578171946 11
71 0.46275900828654204 10
76 0.46201038866523786 10
81 0.4598490609795047 10
86 0.4640143091453645 10
91 0.46357357234919744 10
96 0.4736341906312313 10
Max accuracy: 1 0.5136758819146736


## SVM

In [81]:
import warnings
warnings.filterwarnings('ignore')

In [82]:
pca_components = 3

In [83]:
def classify_nusvm_with_xvalid(data_pp_bp, data_pnp_bp, nu, selected_channels, test_index, mul, verbose=True):
    """
    Leaves out patient at test_index and trains a linear SVM classifier on all other patients
    Validates classifier on patient at test_index
    Returns accuracy over all repetitions for the test patient
    """
    
    data_bp = np.concatenate((data_pp_bp, data_pnp_bp))
    
    test_is_pp, test_label = test_setup(test_index, len(data_pp_bp))
    test_p, train_p = get_train_test(data_bp, test_index)
    train_p_separated = np.vstack(train_p)
    pp_train_len, pnp_train_len = get_pp_pnp_length(pp_count, pnp_count, len(test_p), test_is_pp)
    
    train = pca(ravel_all_trials(train_p_separated, selected_channels) * mul, n_components=pca_components)
    test = pca(ravel_all_trials(test_p, selected_channels) * mul, n_components=pca_components)
    
    labels = [1] * pp_train_len + [0] * pnp_train_len
    test_labels = [test_label] * len(test)
    
    if verbose:
        print('Test index', test_index, 'Preparing to classify set of', pp_train_len, 'PP and', pnp_train_len, 'PNP.')
    
    clas = svm.NuSVC(nu=nu, kernel='linear')
    clas.fit(train, labels)
    train_acc = clas.score(train, labels)
    test_acc = clas.score(test, test_labels)
    
    if verbose:
        print('Train score:', train_acc, '  Test score:', test_acc)
    
    return test_acc
    

In [84]:
total_score = 0
patients_correct = 0
for i in range(len(pp_and_pnp_bp)):
    score = experiments.classify_nusvm_with_xvalid(pp_rh_bp, pnp_rh_bp, 0.8585, [0, 1, 1, 2, 3, 3, 5, 12, 13, 23, 30, 52, 57], i, pca_components=3)
    total_score += score
    if score > 0.5:
        patients_correct += 1
    

print(total_score/len(pp_and_pnp_bp))
print('Correctly labeled', patients_correct, 'out of', len(pp_and_pnp_bp))

Test index 0 Preparing to classify set of 473 PP and 473 PNP.
Train score: 0.6807610993657506   Test score: 0.9649122807017544
Test index 1 Preparing to classify set of 473 PP and 473 PNP.
Train score: 0.7071881606765328   Test score: 1.0
Test index 2 Preparing to classify set of 471 PP and 473 PNP.
Train score: 0.6917372881355932   Test score: 0.5254237288135594
Test index 3 Preparing to classify set of 476 PP and 473 PNP.
Train score: 0.6891464699683878   Test score: 0.9814814814814815
Test index 4 Preparing to classify set of 479 PP and 473 PNP.
Train score: 0.7121848739495799   Test score: 1.0
Test index 5 Preparing to classify set of 472 PP and 473 PNP.
Train score: 0.6835978835978836   Test score: 0.9827586206896551
Test index 6 Preparing to classify set of 474 PP and 473 PNP.
Train score: 0.7286166842661035   Test score: 1.0
Test index 7 Preparing to classify set of 498 PP and 473 PNP.
Train score: 0.713697219361483   Test score: 1.0
Test index 8 Preparing to classify set of 478

In [102]:
file = open('all_results/bandpower_pca_svm_results.csv', 'a', newline='')
n_components = pca_components
name = 'Bandpower + PCA + SVM'
notes = 'freq bands 4-8,8-13,13-30 over 375-500, nu='
notes_c = ', n_components='

previous_channels=[11, 36, 52]
nu = 0.8

max_acc = {'index': 0, 'value': 0}
for channel in range(61):    
    total_score = 0
    correct_patients = 0
    for i in range(len(pp_and_pnp_bp)):
        score = classify_nusvm_with_xvalid(pp_rh_bp, pnp_rh_bp, nu, previous_channels + [channel], i, mul, verbose=False)
        total_score += score
        if score > 0.5:
            correct_patients += 1
        
    avg_score = total_score/len(pp_and_pnp_bp)
    print(channel, avg_score, correct_patients)
    
    log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'RH', str(previous_channels + [channel]), notes + str(nu) + notes_c + str(n_components))
    
    if avg_score > max_acc['value']:
        max_acc['index'] = channel
        max_acc['value'] = avg_score
        

file.close()
        
print('Max accuracy:', max_acc['index'], max_acc['value'])

0 0.8583186562669167 16
1 0.8563693385281255 16
2 0.8583186562669167 16
3 0.8525456768097268 16
4 0.8428976586317629 16
5 0.9081838305745531 17
6 0.9091113333753832 17
7 0.9109502352142852 17
8 0.8427616012361775 16
9 0.853512579773 16
10 0.8583936300261009 16
11 0.8583561431465089 16
12 0.8613176066342881 16
13 0.908026258606098 17
14 0.8564443122873096 16
15 0.9080637454856901 17
16 0.8535203356791226 16
17 0.8476348955831562 16
18 0.9128245791938924 17
19 0.9079887717265058 17
20 0.8612426328751038 16
21 0.8992489135895577 17
22 0.8401706585363418 16
23 0.9038950774008757 17
24 0.8442425423138455 16
25 0.9067891915795573 17
26 0.844710918974969 16
27 0.8514055586097173 16
28 0.8544794827362733 16
29 0.8533329012811618 16
30 0.8584530919730403 16
31 0.9089259437163094 17
32 0.8564894236169882 16
33 0.9099558105356501 17
34 0.8515710179403311 16
35 0.9080637454856901 17
36 0.9119700054133596 17
37 0.9000040663733795 17
38 0.8563394760986196 16
39 0.889342280756966 17
40 0.850201724465

#### Cross validate over multiple nu values

In [105]:
file = open('all_results/bandpower_pca_svm_results.csv', 'a', newline='')
n_components = pca_components
name = 'Bandpower + PCA + SVM'
notes = 'freq bands 4-8,8-13,13-30 over 375-500 nu='
notes_c = ', n_components='

channels = [11, 36, 52]

max_acc = {'index': 0, 'value': 0}
for param in np.arange(0.5, 0.875, 0.01):    
    total_score = 0
    correct_patients = 0
    for i in range(len(pp_and_pnp_bp)):
        score = classify_nusvm_with_xvalid(pp_rh_bp, pnp_rh_bp, param, channels, i, mul, verbose=False)
        total_score += score
        if score > 0.5:
            correct_patients += 1
        
    avg_score = total_score/len(pp_and_pnp_bp)
    print(param, avg_score, correct_patients)
    
    log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'RH', str(channels), notes + str(param) + notes_c + str(n_components))
    
    if avg_score > max_acc['value']:
        max_acc['index'] = param
        max_acc['value'] = avg_score

file.close()
        
print('Max accuracy:', max_acc['index'], max_acc['value'])

0.5 0.5403312814798864 10
0.51 0.5924661918483872 11
0.52 0.48607394191693915 9
0.53 0.43051838636138373 8
0.54 0.49289655400270926 9
0.55 0.5416294974724948 10
0.56 0.8135593220338982 15
0.5700000000000001 0.6498166319754187 12
0.5800000000000001 0.7562083422452454 14
0.5900000000000001 0.47962744762345955 10
0.6000000000000001 0.44026497505534073 8
0.6100000000000001 0.4392903161859451 8
0.6200000000000001 0.4265447771246165 8
0.6300000000000001 0.39638387411052795 8
0.6400000000000001 0.49090974938432563 9
0.6500000000000001 0.4841246241781478 9
0.6600000000000001 0.22584002378828427 4
0.6700000000000002 0.1698042593333128 4
0.6800000000000002 0.3176447565386013 7
0.6900000000000002 0.23907225691347014 5
0.7000000000000002 0.2905481689156979 6
0.7100000000000002 0.34238609706941553 7
0.7200000000000002 0.39696699375557537 8
0.7300000000000002 0.34433541480820695 7
0.7400000000000002 0.11821455710840188 3
0.7500000000000002 0.19007044203143744 4
0.7600000000000002 0.30242508342419133

In [106]:
file.close()